## 1. Ranking base on book-dataset

In [1]:
import warnings
import pandas as pd

warnings.filterwarnings("ignore")
cwd = r"/kaggle/input/book-recommendation-dataset"
book_df = pd.read_csv(f"{cwd}/Books.csv")
rtig_df = pd.read_csv(f"{cwd}/Ratings.csv")
user_df = pd.read_csv(f"{cwd}/Users.csv")
print("Books dataset")
display(book_df.head())
print("Ratings dataset")
display(rtig_df.head())
print("Users dataset")
display(user_df.head())

Books dataset


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


Ratings dataset


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


Users dataset


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


### 1.1. Top book / author / publisher has highest rating
#### 1.1.1. Top book has highest ratings
Must follow these conditions
- Has number of interaction is greater than 15
-`median-rate` is greater than 8 and `average-rate` is at least 6.5
- Sorting by `average-rate`

In [2]:
tempdf = pd.pivot_table(rtig_df, index='ISBN', values='Book-Rating', aggfunc=['mean', 'count', 'max', 'median'])
tempdf.columns = ['avg_rates', 'count_rates', 'max_rated', 'median_rated']
tempdf = tempdf[(tempdf['count_rates'] > 15) & (tempdf['median_rated'] > 8) & (tempdf['avg_rates'] > 6.5)]
pd.merge_ordered(left=book_df[['ISBN', 'Book-Title', 'Book-Author', 'Publisher']], 
                 right=tempdf.reset_index(),
                 on='ISBN', how='right'
                ).dropna().sort_values(by='avg_rates', ascending=False)

,ISBN,Book-Title,Book-Author,Publisher,avg_rates,count_rates,max_rated,median_rated
10,1844262553,Free,Paul Vincent,Upfront Publishing,7.962963,54,10,9.0
8,0836213122,There's Treasure Everywhere--A Calvin and Hobb...,Bill Watterson,Andrews McMeel Publishing,7.882353,17,10,9.0
11,8478886451,Harry Potter y el cÃ¡liz de fuego,J. K. Rowling,Lectorum Publications,7.875000,16,10,8.5
1,0060294698,All-American Girl,Meg Cabot,HarperCollins,7.235294,17,10,9.0
4,0394800389,Fox in Socks (I Can Read It All by Myself Begi...,Dr. Seuss,Random House Children's Books,7.210526,19,10,10.0
0,0060256672,Where the Sidewalk Ends : Poems and Drawings,Shel Silverstein,HarperCollins,7.121212,33,10,10.0
2,0064440508,A Kiss for Little Bear,Else Holmelund Minarik,HarperTrophy,7.100000,20,10,8.5
7,0553274325,Johnny Got His Gun,Dalton Trumbo,Bantam Books,6.750000,32,10,9.0
9,0836221362,It's A Magical World: A Calvin and Hobbes Coll...,Bill Watterson,Andrews McMeel Publishing,6.642857,28,10,9.0
5,0439064864,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,Scholastic,6.611765,170,10,9.0


#### 1.1.2 Top authors has higest rating, interactions
- Number of interactions (of each books) is at least 30
- avg(rate) for each books is at-least 6.0

In [3]:
tempdf = pd.merge_ordered(left=book_df[['ISBN', 'Book-Title', 'Book-Author']], 
                 right=rtig_df,
                 on='ISBN', how='right'
                ).dropna()
tempdf = pd.pivot_table(tempdf, index=['Book-Author','ISBN', 'Book-Title'], 
               values='Book-Rating', aggfunc=['mean', 'count', 'max', 'median'])
tempdf.columns = ['avg_rates', 'count_rates', 'max_rated', 'median_rated']
tempdf = tempdf[(tempdf['count_rates'] > 30) & (tempdf['median_rated'] > 7) & (tempdf['avg_rates'] > 6)]
tempdf.sort_values(by=['count_rates', 'avg_rates'], ascending=[False, False])

avg_rates  \
Book-Author      ISBN       Book-Title                                                      
J. K. Rowling    0439136350 Harry Potter and the Prisoner of Azkaban (Book 3)    6.467005   
                 0439139597 Harry Potter and the Goblet of Fire (Book 4)         6.541237   
                 0439064864 Harry Potter and the Chamber of Secrets (Book 2)     6.611765   
                 0590353403 Harry Potter and the Sorcerer's Stone (Book 1)       6.363095   
Nick Bantock     0877017883 Griffin &amp; Sabine: An Extraordinary Correspo...   6.041667   
J. R. R. Tolkien 0618002227 The Fellowship of the Ring (The Lord of the Rin...   6.206349   
Stephen King     0385199570 The Stand (The Complete and Uncut Edition)           6.175439   
Paul Vincent     1844262553 Free                                                 7.962963   
J. R. R. Tolkien 0618002235 The Two Towers (The Lord of the Rings, Part 2)       6.230769   
Shel Silverstein 0060248025 Falling Up                                           6.921053   
C. S. Lewis      0064471063 The Horse and His Boy                                6.216216   
Robert N. Munsch 0920668372 Love You Forever                                     6.444444   
Eoin Colfer      0786808551 The Arctic Incident (Artemis Fowl, Book 2)           6.088235   
Shel Silverstein 0060256672 Where the Sidewalk Ends : Poems and Drawings         7.121212   
Dalton Trumbo    0553274325 Johnny Got His Gun                                   6.750000   

                                                                                count_rates  \
Book-Author      ISBN       Book-Title                                                        
J. K. Rowling    0439136350 Harry Potter and the Prisoner of Azkaban (Book 3)           197   
                 0439139597 Harry Potter and the Goblet of Fire (Book 4)                194   
                 0439064864 Harry Potter and the Chamber of Secrets (Book 2)            170   
                 0590353403 Harry Potter and the Sorcerer's Stone (Book 1)              168   
Nick Bantock     0877017883 Griffin &amp; Sabine: An Extraordinary Correspo...           72   
J. R. R. Tolkien 0618002227 The Fellowship of the Ring (The Lord of the Rin...           63   
Stephen King     0385199570 The Stand (The Complete and Uncut Edition)                   57   
Paul Vincent     1844262553 Free                                                         54   
J. R. R. Tolkien 0618002235 The Two Towers (The Lord of the Rings, Part 2)               39   
Shel Silverstein 0060248025 Falling Up                                                   38   
C. S. Lewis      0064471063 The Horse and His Boy                                        37   
Robert N. Munsch 0920668372 Love You Forever                                             36   
Eoin Colfer      0786808551 The Arctic Incident (Artemis Fowl, Book 2)                   34   
Shel Silverstein 0060256672 Where the Sidewalk Ends : Poems and Drawings                 33   
Dalton Trumbo    0553274325 Johnny Got His Gun                                           32   

                                                                                max_rated  \
Book-Author      ISBN       Book-Title                                                      
J. K. Rowling    0439136350 Harry Potter and the Prisoner of Azkaban (Book 3)          10   
                 0439139597 Harry Potter and the Goblet of Fire (Book 4)               10   
                 0439064864 Harry Potter and the Chamber of Secrets (Book 2)           10   
                 0590353403 Harry Potter and the Sorcerer's Stone (Book 1)             10   
Nick Bantock     0877017883 Griffin &amp; Sabine: An Extraordinary Correspo...         10   
J. R. R. Tolkien 0618002227 The Fellowship of the Ring (The Lord of the Rin...         10   
Stephen King     0385199570 The Stand (The Complete and Uncut Edition)                 10   
Paul Vincent     1844262553 Free             

#### 1.1.3. Top publishers has good interactions
- 1st priority is for the total book-published
- next is the ranking of book

In [4]:
tempdf = pd.merge_ordered(left=book_df[['ISBN', 'Book-Title', 'Publisher']],
                 right=rtig_df,
                 on='ISBN', how='right'
                ).dropna()
tempdf = tempdf.groupby('Publisher').agg({'ISBN': lambda x: x.nunique(), 
                                          'Book-Rating': ['mean', 'median', 'max']})
tempdf.columns = ['count_book(distinct)', 'avg_book_rate', 'median_book_rate', 'max_rate']
tempdf[(tempdf['count_book(distinct)'] > 50) 
           & (tempdf['avg_book_rate'] > 5) 
           & (tempdf['median_book_rate'] > 6)].sort_values(
                                                by=['count_book(distinct)', 'avg_book_rate'], 
                                                ascending=[False, False])

,count_book(distinct),avg_book_rate,median_book_rate,max_rate
Publisher,,,,
Five Star (ME),183,7.884058,9.0,10
DC Comics,157,5.913043,8.0,10
Authorhouse,155,5.285714,7.0,10
Tokyopop,142,6.277419,8.0,10
1stBooks Library,129,5.687117,8.0,10
TokyoPop,99,6.570732,8.0,10
Dark Horse Comics,97,5.107595,7.0,10
Poisoned Pen Press,83,5.664062,8.0,10
VIZ LLC,80,6.382353,8.0,10


### 1.2. Top authors has highest affected

In [5]:
tempdf = pd.merge_ordered(left=book_df[['ISBN', 'Book-Title', 'Book-Author']],
                 right=rtig_df,
                 on='ISBN', how='right'
                ).dropna()
tempdf = tempdf.groupby('Book-Author').agg({'ISBN': lambda x: x.nunique(), 
                                            'Book-Rating': ['mean', 'median', 'max']})
tempdf.columns = ['count_book(distinct)', 'avg_book_rate', 'median_book_rate', 'max_rate']
tempdf[(tempdf['count_book(distinct)'] > 35)
           & (tempdf['avg_book_rate'] > 5) 
           & (tempdf['median_book_rate'] > 6)].sort_values(
                                                by=['count_book(distinct)', 'avg_book_rate'], 
                                                ascending=[False, False])

,count_book(distinct),avg_book_rate,median_book_rate,max_rate
Book-Author,,,,
J. R. R. Tolkien,173,5.165644,7.0,10
J. K. Rowling,77,5.385832,8.0,10
Harlan Ellison,58,6.134021,8.0,10
Herge,52,5.519737,8.0,10
Clamp,50,6.513333,9.0,10
Rumiko Takahashi,43,6.445783,8.0,10
Pablo Neruda,43,5.241758,7.0,10
Bill Watterson,42,5.561818,8.0,10
Dr. Seuss,40,5.121294,8.0,10


Noting that we used `'ISBN': lambda x: x.nunique()` is aimed to count the unique books

In [6]:
tempdf = pd.merge_ordered(left=book_df[['ISBN', 'Book-Title', 'Book-Author']],
                 right=rtig_df,
                 on='ISBN', how='right'
                ).dropna()
print(tempdf.loc[tempdf['Book-Author'] == 'Naoko Takeuchi', 'Book-Title'].unique())
len(tempdf.loc[tempdf['Book-Author'] == 'Naoko Takeuchi', 'Book-Title'].unique())

['Sailor Moon: Friends &amp; Foes' 'Sailor Moon (Sailor Moon)'
 'Sailor Moon, Vol. 3' 'Sailor Moon Supers, Vol. 1' 'Sailor Moon, Vol. 4'
 'Sailor Moon, Vol. 5' 'Sailor Moon Supers, Vol. 2'
 'Sailor Moon Supers, Vol. 3' 'Sailor Moon 6' 'Sailor Moon Supers #4'
 'Sailor Moon #7' 'Sailor Moon Vol. 8' 'Sailor Moon Stars 1'
 'Sailor Moon Vol. 9' 'Sailor Moon Stars #2' 'Sailor Moon Stars # 3'
 'Sailor Moon #10' 'Sailor Moon #11'
 'Sailor Moon, tome 1 : MÃ?Â©tamorphose'
 'Sailor Moon, tome 3 : Les JusticiÃ?Â¨res de la lune'
 "Sailor Moon, tome 4 : Le cristal d'argent"
 'Sailor Moon, tome 5 : La gardienne du temps'
 'Sailor Moon, tome 6 : La PlanÃ?Â¨te NÃ?Â©mÃ?Â©sis'
 'Sailor Moon, tome 7 : Black lady'
 'Sailor Moon, tome 8 : Le LycÃ?Â©e infini'
 'Sailor Moon, tome 9 : Uranus et Neptune'
 'Sailor Moon, tome 10 : Sailor saturne'
 'Sailor Moon, tome 11 : La Princesse Kaguya'
 'Sailor Moon, tome 12 : PÃ?Â©gase' 'Sailor Moon, tome 13 : HÃ?Â©lios'
 'Sailor Moon, tome 14 : Le royaume Elusion'
 'Sailo

37

### 1.3. Top countries has highest book-interactions
Firstly, we need to extract the `country` in the `user_df`

In [7]:
user_df['Country'] = user_df['Location'].apply(lambda x: x.split(',')[-1].title())
user_df = user_df[user_df['Country'] != '']
user_df.head()

,User-ID,Location,Age,Country
0,1,"nyc, new york, usa",NaN,Usa
1,2,"stockton, california, usa",18.0,Usa
2,3,"moscow, yukon territory, russia",NaN,Russia
3,4,"porto, v.n.gaia, portugal",17.0,Portugal
4,5,"farnborough, hants, united kingdom",NaN,United Kingdom


Now,

In [8]:
tempdf = pd.merge_ordered(left=book_df[['ISBN', 'Book-Title', 'Book-Author']],
                 right=rtig_df,
                 on='ISBN', how='right'
                ).dropna()
tempdf = pd.merge_ordered(left=tempdf, right=user_df,on='User-ID',how='left').dropna()
tempdf = tempdf.groupby('Country').agg({
                                        'User-ID': 'count',
                                        'Book-Title': lambda x: x.nunique(),
                                        'Book-Rating': ['mean', 'max']
                                    })
tempdf.columns = ['total_intereactions', 'distinct_books_reacted', 'avg_book_rate', 'max_rate']
tempdf[(tempdf['total_intereactions'] > 500) & (tempdf['distinct_books_reacted'] > 100) & (tempdf['avg_book_rate'] > 2.5)].sort_values(
                by=['total_intereactions', 'distinct_books_reacted'], 
                ascending=[False, False])

,total_intereactions,distinct_books_reacted,avg_book_rate,max_rate
Country,,,,
Usa,553788,153213,2.611707,10
Canada,65429,35613,2.936802,10
United Kingdom,26848,16218,3.285906,10
Germany,19638,10926,3.246410,10
Australia,14044,9918,2.781401,10
Spain,13070,7745,3.810559,10
France,9943,5650,3.301016,10
Portugal,6564,4100,3.898842,10
Netherlands,3913,2913,2.613340,10


## 2. IBM ranking-base

In [9]:
import sqlite3, warnings
from sqlalchemy import create_engine

warnings.filterwarnings("ignore")
engine = create_engine('sqlite:////kaggle/input/movies/movies', echo=True)
connection = engine.connect()

query1 = """ SELECT * FROM articles_data """
query2 = """ SELECT * FROM user_interactions """
art_df = pd.read_sql(query1, con=connection.connection)
user_df = pd.read_sql(query2, con=connection.connection)
display(art_df.head())
display(user_df.head())

,doc_body,doc_description,doc_full_name,doc_status,article_id
0,Skip navigation Sign in SearchLoading...\r\n\r...,Detect bad readings in real time using Python ...,Detect Malfunctioning IoT Sensors with Streami...,Live,0
1,No Free Hunch Navigation * kaggle.com\r\n\r\n ...,"See the forest, see the trees. Here lies the c...",Communicating data science: A guide to present...,Live,1
2,☰ * Login\r\n * Sign Up\r\n\r\n * Learning Pat...,Here’s this week’s news in Data Science and Bi...,"This Week in Data Science (April 18, 2017)",Live,2
3,"DATALAYER: HIGH THROUGHPUT, LOW LATENCY AT SCA...",Learn how distributed DBs solve the problem of...,DataLayer Conference: Boost the performance of...,Live,3
4,Skip navigation Sign in SearchLoading...\r\n\r...,This video demonstrates the power of IBM DataS...,Analyze NY Restaurant data using Spark in DSX,Live,4


,article_id,title,email
0,1430.0,"using pixiedust for fast, flexible, and easier...",ef5f11f77ba020cd36e1105a00ab868bbdbf7fe7
1,1314.0,healthcare python streaming application demo,083cbdfa93c8444beaa4c5f5e0f5f9198e4f9e0b
2,1429.0,use deep learning for image classification,b96a4f2e92d8572034b1e9b28f9ac673765cd074
3,1338.0,ml optimization using cognitive assistant,06485706b34a5c9bf2a0ecdac41daf7e7654ceb7
4,1276.0,deploy your python model as a restful api,f01220c46fc92c6e6b161b1849de11faacd7ccb2


In these cases, the popularity of an article can really only be based on how often an article was interacted with.
- What is the distribution of how many `articles` a user interacts with in the dataset?

In [10]:
df_users_artc = user_df.groupby('article_id').count()[['email']]
df_users_artc.columns = ['n_interacts']
df_users_artc.sort_values(by='n_interacts', ascending=False).head(10)

,n_interacts
article_id,
1429.0,937
1330.0,927
1431.0,671
1427.0,643
1364.0,627
1314.0,614
1293.0,572
1170.0,565
1162.0,512


- Next, top email that interacts to the `arcticle` [Noting that each `arcticle_id` will receive many `email`]

In [11]:
df_artc_user = user_df.groupby('email').count()[['article_id']]
df_artc_user.columns = ['n_interacts']
df_artc_user.sort_values(by='n_interacts', ascending=False).head(10)

,n_interacts
email,
2b6c0f514c2f2b04ad3c4583407dccd0810469ee,364
77959baaa9895a7e2bdc9297f8b27c1b6f2cb52a,363
2f5c7feae533ce046f2cb16fb3a29fe00528ed66,170
a37adec71b667b297ed2440a9ff7dad427c7ac85,169
8510a5010a5d4c89f5b07baac6de80cd12cfaf93,160
f8c978bcf2ae2fb8885814a9b85ffef2f54c3c76,158
284d0c17905de71e209b376e3309c0b08134f7e2,148
18e7255ee311d4bd78f5993a9f09538e459e3fcc,147
d9032ff68d0fd45dfd18c0c5f7324619bb55362c,147


In [12]:
# Remove any rows that have the same article_id - only keep the first
art_df.drop_duplicates(subset = 'article_id', inplace = True)
print(art_df.shape)
art_df.nunique()

(1051, 5)


doc_body           1031
doc_description    1019
doc_full_name      1051
doc_status            1
article_id         1051
dtype: int64

#### Top title has most views


In [13]:
tempdf = user_df.groupby('title').agg({'article_id': 'count',
                                       'email': lambda x: x.nunique()
                             })
tempdf.columns = ['count', 'count_distinct']
tempdf.sort_values(by='count_distinct', ascending=False).head(10)

,count,count_distinct
title,,
insights from new york car accident reports,927,467
use deep learning for image classification,937,397
predicting churn with the spss random tree algorithm,627,388
healthcare python streaming application demo,614,345
total population by country,465,329
visualize car data with brunel,671,320
customer demographics and sales,473,314
"use xgboost, scikit-learn & ibm watson machine learning apis",643,308
analyze accident reports on amazon emr spark,433,299
